In [ ]:
import cx_Oracle
import datastory
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils

importlib.reload(pandas_utils)

In [ ]:
import dataverk_tools.themes.navdesign as theme
import dataverk_tools.themes.navdesign.plotly_template as template
pio.templates.default = template

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()
resource_name = f"projects/knada-gcp/secrets/vebjorn-rekkebo-bac1/versions/latest"
secret = secrets.access_secret_version(name=resource_name)
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

ORACLE_HOST = '10.53.136.15'
ORACLE_PORT = '1521'
ORACLE_SERVICE = 'pen'

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

# Antall krav, førstegangsbehandling innland/utland

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_krav = pandas_utils.pandas_from_sql('../sql/kravcount.sql', con)
con.close()

In [ ]:
df_krav.head()

In [ ]:
df = df_krav.copy()
df["MANED"] = df.MANED.astype('string').apply(lambda x: pandas_utils.add_zero_to_mnd(x))
df["AR_MANED"] = df.AR_MANED.apply(lambda x: pandas_utils.add_zero_to_aar_mnd(x))

In [ ]:
df.head()

In [ ]:
figs = {}

In [ ]:
figs["gjenlev_mmd"] = px.bar(df[df.STONAD == "Gjenlevendeytelse"], "AR_MANED", "ANTALL", color="KRAVTYPE", labels={"AR_MANED": "Måned", "ANTALL":"Antall", "KRAVTYPE": "Kravtype"})

In [ ]:
figs["barnep_mmd"] = px.bar(df[df.STONAD == "Barnepensjon"], "AR_MANED", "ANTALL", color="KRAVTYPE", labels={"AR_MANED": "Måned", "ANTALL":"Antall", "KRAVTYPE": "Kravtype"})

In [ ]:
df_ar = (df
 .groupby(["AR", "STONAD", "KRAVTYPE"], as_index=False)
 .ANTALL
 .sum()
)
df_ar["AR"] = df_ar["AR"].apply(str)

In [ ]:
figs["barnep_ar"] = px.bar(df_ar[df_ar.STONAD == "Barnepensjon"], "AR", "ANTALL", color="KRAVTYPE", labels={"AR": "År", "ANTALL":"Antall", "KRAVTYPE": "Kravtype"})

In [ ]:
figs["gjenlev_ar"] = px.bar(df_plot[df_plot.STONAD == "Gjenlevendeytelse"], "AR", "ANTALL", color="KRAVTYPE", labels={"AR": "År", "ANTALL":"Antall", "KRAVTYPE": "Kravtype"})

# Ledetid

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_ledetid = pandas_utils.pandas_from_sql('../sql/ledetid.sql', con)
con.close()

In [ ]:
df = df_ledetid.copy()

In [ ]:
df.head()

In [ ]:
labels = {
        "GJ_DAGER": "Gjennomsnittlig antall dager",
        "AR": "År",
        "STOENAD": "Stønad",
        "KRAVTYPE": "Kravtype",
        "MED_DAGER": "Median av antall dager",
        "ANTALL": "Antall krav"
           }

In [ ]:
df_bp = df[df.STOENAD == "Barnepensjon"]
df_gjenlev = df[df.STOENAD == "Gjenlevendeytelse"]

In [ ]:
figs["barnep_gj_dager"] = px.bar(
    df_bp[df_bp.ANTALL >= 5], 
    "AR", 
    "GJ_DAGER", 
    color="KRAVTYPE", 
    barmode="group",
    hover_data=df_bp.columns[[2,3,4,5]],
    labels=labels
    )

figs["barnep_gj_dager"]

In [ ]:
figs["barnep_med_dager"] = px.bar(
    df_bp[df_bp.ANTALL >= 5], 
    "AR", 
    "MED_DAGER", 
    color="KRAVTYPE", 
    barmode="group", 
    hover_data=df_bp.columns[[2,3,4,5]],
    labels=labels
    )

In [ ]:
figs["gjenlev_gj_dager"] = px.bar(
    df_gjenlev[df_gjenlev.ANTALL >= 5], 
    "AR", 
    "GJ_DAGER", 
    color="KRAVTYPE", 
    barmode="group", 
    hover_data=df_gjenlev.columns[[2,3,4,5]],
    labels=labels
    )

In [ ]:
figs["gjenlev_med_dager"] = px.bar(
    df_gjenlev[df_gjenlev.ANTALL >= 5], 
    "AR", 
    "MED_DAGER", 
    color="KRAVTYPE", 
    barmode="group", 
    hover_data=df_gjenlev.columns[[2,3,4,5]],
    labels=labels
    )

In [ ]:
story = datastory.DataStory("Gjenlevendepensjon og barnepensjon")

story.header("Antall opprettede krav fordelt på innland/utland", level=1)

story.header("Gjenlevendepensjon", level=2)
story.header("Førstegangsbehandling gjenlevendepensjon per måned", level=3)
story.plotly(figs["gjenlev_mmd"].to_json())
story.header("Førstegangsbehandling gjenlevendepensjon per år", level=3)
story.plotly(figs["gjenlev_ar"].to_json())

story.header("Barnepensjon", level=2)
story.header("Førstegangsbehandling barnepensjon per måned", level=3)
story.plotly(figs["barnep_mmd"].to_json())
story.header("Førstegangsbehandling barnepensjon per år", level=3)
story.plotly(figs["barnep_ar"].to_json())

story.header("Tid fra mottatt krav til iverksatt vedtak")
story.header("Gjenlevendepensjon", level=2)
story.plotly(figs["gjenlev_gj_dager"].to_json())
story.plotly(figs["gjenlev_med_dager"].to_json())

story.header("Barnepensjon", level=2)
story.plotly(figs["barnep_gj_dager"].to_json())
story.plotly(figs["barnep_med_dager"].to_json())

In [ ]:
story.publish(url="https://nada.intern.nav.no/api")